 ## Decorators

### Decorators for Functions

> * Put simply: decorators wrap a function, modifying its behavior

In [3]:
def do_twice(func):
    def wrapper_do_twice(*args, **kwargs):
        func(*args, **kwargs)
        func(*args, **kwargs)
    return wrapper_do_twice

@do_twice
def greet(name):
    print('Hello {}'.format(name))

In [4]:
greet('ODL')

Hello ODL
Hello ODL


#### @functools.wraps: Preserving information 

In [5]:
greet.__name__

'wrapper_do_twice'

In [7]:
import functools

def do_twice_wrap(func):
    @functools.wraps(func)
    def wrapper_do_twice(*args, **kwargs):
        func(*args, **kwargs)
        return func(*args, **kwargs)
    return wrapper_do_twice

@do_twice_wrap
def greet_wrapped(name):
    print('Hello {}'.format(name))


In [8]:
greet_wrapped('ODL')

Hello ODL
Hello ODL


In [9]:
greet_wrapped.__name__

'greet_wrapped'

#### Simple decorator example 1 (Timing Functions)

In [17]:
import functools
import time

def timer(func):
    """Print the runtime of the decorated function"""
    @functools.wraps(func)
    def wrapper_timer(*args, **kwargs):
        start_time = time.perf_counter()
        value = func(*args, **kwargs)
        end_time = time.perf_counter()
        run_time = end_time - start_time
        print('Finished {!r} in {:.4f} secs'.format(func.__name__, run_time))
        return value
    return wrapper_timer

@timer
def waste_some_time(num_times):
    for _ in range(num_times):
        sum([i **2 for i in range(5000)])

In [26]:
waste_some_time(1)

Finished 'waste_some_time' in 0.0024 secs


#### Simple decorator example 2 (Debugging Code)

In [29]:
import functools

def debug(func):
    """Print the function signature and return value"""
    @functools.wraps(func)
    def wrapper_debug(*args, **kwargs):
        args_repr = [repr(a) for a in args]
        kwargs_repr = ['{}={!r}'.format(k, v) for k, v in kwargs.items()]
        signature = ', '.join(args_repr + kwargs_repr)
        print('Calling {}({})'.format(func.__name__, signature))
        value = func(*args, **kwargs)
        print('{} returned {!r}'.format(func.__name__, value))
        return value
    return wrapper_debug

@debug
def make_greeting(name, age=None):
    if age is None:
        return 'Howdy {}!'.format(name)
    else:
        return 'Whoa {}! {} already, you are growing up!'.format(name, age)

In [31]:
make_greeting('odl', age=20)

Calling make_greeting('odl', age=20)
make_greeting returned 'Whoa odl! 20 already, you are growing up!'


'Whoa odl! 20 already, you are growing up!'

#### Simple decorator example 3 (Registering Plugins)

In [36]:
import random
PLUGINS = dict()

def register(func):
    """Reguster a function as as plug-in"""
    PLUGINS[func.__name__] = func
    return func

@register
def say_hello(name):
    return 'Hello {}'.format(name)

@register
def be_awesome(name):
    return 'Yo {}, together we are the awesomest'.format(name)

def randomly_greet(name):
    greeter, greeter_func = random.choice(list(PLUGINS.items()))
    print('Using {!r}'.format(greeter))
    return greeter_func(name)


In [41]:
print(PLUGINS)
randomly_greet('Alice')

{'say_hello': <function say_hello at 0x0000026FB6935950>, 'be_awesome': <function be_awesome at 0x0000026FB6935AE8>}
Using 'say_hello'


'Hello Alice'

### Decorators for Class

#### built-in class decorators (@staticmethod, @classmethod, @property)

In [53]:
class Circle:
    def __init__(self, radius):
        self._radius = radius
        
    @property
    def radius(self):
        return self._radius
    
    @radius.setter
    def radius(self, value):
        if value >= 0:
            self._radius = value
        else:
            raise ValueError('Radius must be positive')
            
    @property
    def area(self):
        return self.pi() * self.radius**2
    
    def cylinder_volume(self, height):
        return self.area * height
    
    @classmethod
    def unit_circle(cls):
        return cls(1)
    
    @staticmethod
    def pi():
        return 3.141592

In [57]:
c = Circle(5)
c.area

78.5398

#### Decorators with Arguments